# 4-SFT

在预训练阶段后，我们应该能够获得一个下一词预测模型，此时的模型已经掌握了大量的知识。不过，仅仅具备下一词预测能力是不够的，我们希望大模型能够获得问答能力，这一能力便是在有监督微调（Supervised Fine Tuning，SFT）阶段获得的.

在这个笔记本中，我们仅对 SFT 的训练流程进行展示和学习，因此只给出必要的代码片段，如 wandb 和 ddp 不会在此笔记本中涉及.

此笔记本的完整实现见主仓库 `/minimind/train_full_sft.py`

In [1]:
# 导入依赖
import os
import platform
import argparse
import time
import math
import warnings

import pandas as pd
import torch
import torch.nn.functional as F
import torch.distributed as dist
from contextlib import nullcontext

from torch import optim, nn
from torch.nn.parallel import DistributedDataParallel
from torch.utils.data import DataLoader, DistributedSampler
from transformers import AutoTokenizer, AutoModelForCausalLM
from model.model import MiniMindLM
from model.LMConfig import LMConfig
from model.dataset import SFTDataset

In [2]:
warnings.filterwarnings('ignore')

## 可选参数设置

首先，查看训练的可选参数，这些参数在实际使用时通过命令行导入，为了保持笔记本的易用性，选择用 class 进行包装.

In [3]:
class args:
    epochs: int = 5 # 训练轮数，延续 pretrain 基础上微调
    batch_size: int = 2 # pretrain 数据集仅两个样本，设置 batch 为 2
    learning_rate: float = 5e-4 # 学习率
    device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    dtype: str = 'bfloat16' # 16 bit 浮点数：8 bit 指数 + 7 bit 尾数
    # use_wandb: bool = False # 是否使用 wandb 我们不使用
    wandb_project: str = 'MiniMind-Notebook'
    num_workers: int = 1 # 工作进程数
    # ddp：bool = False # 单机多卡
    accumulation_steps: int = 1 # 梯度累积步数
    grad_clip: float = 1.0 # 梯度剪裁
    warmup_iters: int = 0 # 学习率热启动
    log_interval: int = 1 # 每一步打印日志 仅用于观察
    local_rank: int = 1 # device 设备号
    dim: int = 512 # 词嵌入维度 模型超参数
    n_layers: int = 2 # MiniMind Block 数量 模型超参数
    max_seq_len: int = 512 # 序列长度阈值
    use_moe: bool = False # 是否启用混合专家
    data_path: str = './toydata/sft_data.jsonl' # 数据集路径
    save_dir: str = "./output"  # 模型保存目录
    save_weight: str = "minimind_sft"  # checkpoint 文件前缀
    save_interval: int = 1  # 每多少步保存一次模型，0表示不保存 我们这里只展示训练过程（可选择的保存模型，建议先保存）

## 初始化训练

接下来，我们对一些重要模块进行初始化，我们已经了解过，分词器，模型和数据集是大模型的基本组件，我们对其进行初始化.

> 注意 与预训练阶段不同的是 在 sft 阶段 我们实际上是在上一阶段训练获得的模型的基础上修改数据集进行接续训练 因此需要载入上一阶段的模型权重 出于展示的目的 载入权重的代码在此笔记本中只作展示 并不执行

In [4]:
def init_model(lm_config):
    tokenizer = AutoTokenizer.from_pretrained('./model/minimind_tokenizer')
    model = MiniMindLM(lm_config).to(args.device)
    moe_path = '_moe' if lm_config.use_moe else ''
    ckp = f'./output/minimind_pretrain_{lm_config.dim}{moe_path}.pth' # 指示上一阶段训练保存的模型文件位置
    state_dict = torch.load(ckp, map_location=args.device) # 载入模型状态字典
    model.load_state_dict(state_dict, strict=False) # 装入模型
    print(f'LLM总参数量：{sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6:.3f} 百万')
    model = model.to(args.device)
    return model, tokenizer

In [5]:
lm_config = LMConfig(dim=args.dim, n_layers=args.n_layers, max_seq_len=args.max_seq_len, use_moe=args.use_moe)
model, tokenizer = init_model(lm_config)

# 准备数据集和数据加载器
train_ds = SFTDataset(args.data_path, tokenizer, max_length=lm_config.max_seq_len)

train_loader = DataLoader(
    train_ds,
    batch_size=args.batch_size,
    pin_memory=True,
    drop_last=False,
    shuffle=False,
    num_workers=args.num_workers,
)

print(f'模型位于设备：{model.device}, 词表长度：{tokenizer.vocab_size}, DataLoader：{train_loader}')

LLM总参数量：8.915 百万
模型位于设备：cuda:0, 词表长度：6400, DataLoader：<torch.utils.data.dataloader.DataLoader object at 0x0000020E7FB04910>


In [6]:
loader = iter(train_loader)
print(f'打印一个 iter 的数据:\n{next(loader)}\n')
print(f'数据集大小：{len(train_ds)}, DataLoader 大小：{len(loader)}')

打印一个 iter 的数据:
[tensor([[  1,  85, 736,  ...,   0,   0,   0],
        [  1,  85, 736,  ...,   0,   0,   0]]), tensor([[ 85, 736, 201,  ...,   0,   0,   0],
        [ 85, 736, 201,  ...,   0,   0,   0]]), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])]

数据集大小：2, DataLoader 大小：1


我们发现，train loader 的每一个 iter 都包含一个长度为 3 的张量列表，这是因为 train_dataset 每一次取数据都会返回三个张量，分别为:

- 样本 X: 包含 \<bos> 在内的输入 conversation
- 标签 Y: 包含 \<eos> 在内的输出 conversation
- 掩码 loss_mask: 指示需要计算损失的 token 位置

由于我们的数据集只有两条数据，而 batch size 设置为 2，因此我们的 dataloader 只有一个 iter.

## 启动训练

训练一个深度学习模型，还涉及到了优化器，损失函数和学习率调度. 接下来，我们查看 MiniMind 训练部分的代码，并进行一轮简单的训练.

> 不难发现 pretrain 阶段和 sft 阶段的训练主体差不多 因为这两个阶段的差异体现在数据集格式 而数据集在经过 chat template 格式化后差异小了很多

In [ ]:
# 学习率调度方面 采用余弦退火学习率
def get_lr(current_step, total_steps, lr):
    return lr / 10 + 0.5 * lr * (1 + math.cos(math.pi * current_step / total_steps))

# 优化器方面 选择 AdamW 优化器 并在混精度场景下创建 scaler 进行梯度缩放避免数值下溢
scaler = torch.amp.GradScaler('cuda', enabled=(args.dtype in ['float16', 'bfloat16']))  # 专门解决混合精度训练中的数值下溢问题
optimizer = optim.AdamW(model.parameters(), lr=args.learning_rate)  # AdamW 优化器

device_type = "cuda" if "cuda" in args.device else "cpu"
print(f'设备类型：{device_type}')
# 根据指定的数据类型设置混精度训练的 dtype，以下步骤为不可缺少的混精度训练准备工作
if args.dtype == 'bfloat16':
    amp_dtype = torch.bfloat16
elif args.dtype == 'float16':
    amp_dtype = torch.float16
else:
    amp_dtype = torch.float32  # 默认为 FP32
print(f'使用混精度训练，数据类型：{amp_dtype}')
# 在 cuda 上启动混精度训练，否则空白上下文
autocast_ctx = nullcontext() if device_type == "cpu" else torch.amp.autocast(device_type='cuda', dtype=amp_dtype) 

设备类型：cuda
使用混精度训练，数据类型：torch.bfloat16


接下来，我们来看看 MiniMind 的训练函数

In [ ]:
def train_epoch(epoch, loader, iters, start_step=0, wandb=None):
    start_time = time.time()
    for step, (X, Y, loss_mask) in enumerate(loader, start=start_step + 1):
        # 将输入数据移动到指定设备
        X = X.to(args.device)
        Y = Y.to(args.device)
        loss_mask = loss_mask.to(args.device)
        lr = get_lr(epoch * iters + step, args.epochs * iters, args.learning_rate)

        # 更新优化器的学习率
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        with autocast_ctx:
            res = model(input_ids=X)  # 前向推理
            logits = res.logits  # [batch, seq_len, vocab_size]

            # 对logits/labels/loss_mask做同步截断（去掉最后1位，避免预测未来token，与pretrain不同的地方）
            shift_logits = logits[..., :-1, :].contiguous()  # [batch, seq_len-1, vocab]
            shift_labels = Y[..., 1:].contiguous()      # [batch, seq_len-1]
            shift_loss_mask = loss_mask[..., 1:].contiguous() # [batch, seq_len-1] 

            loss_fct = nn.CrossEntropyLoss(reduction='none')
            raw_loss = loss_fct(
                shift_logits.reshape(-1, shift_logits.size(-1)),  # [batch*(seq_len-1), vocab]
                shift_labels.reshape(-1)                          # [batch*(seq_len-1)]
            )
            # 应用损失掩码
            shift_loss_mask = shift_loss_mask.reshape(-1)  # [batch*(seq_len-1)]
            masked_loss = (raw_loss * shift_loss_mask).sum() / (shift_loss_mask.sum() + 1e-8)
            # 加上moe的辅助损失（若无moe则aux_loss为0）
            total_loss = masked_loss + (res.aux_loss if res.aux_loss is not None else 0.0)
            # 梯度累积：损失除以累积步数  相当于在显存受限的情况下模拟更大的 batch size
            total_loss = total_loss / args.accumulation_steps
            
            # 梯度累积：损失归一化
            total_loss = total_loss / args.accumulation_steps

        scaler.scale(total_loss).backward()

        if (step + 1) % args.accumulation_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)

            scaler.step(optimizer)
            scaler.update()

            optimizer.zero_grad(set_to_none=True)

        if step % args.log_interval == 0 or step == iters - 1:
            spend_time = time.time() - start_time  # 计算已用时间
            current_loss = total_loss.item() * args.accumulation_steps  # 恢复实际损失值
            current_aux_loss = res.aux_loss if res.aux_loss is not None else 0.0  # 辅助损失
            current_logits_loss = current_loss - current_aux_loss  
            current_lr = optimizer.param_groups[-1]['lr']
            # 计算剩余时间
            eta_seconds = (spend_time / step) * (iters - step) if step > 0 else 0
            eta_min = eta_seconds // 60
            print(
                f'Epoch:[{epoch + 1}/{args.epochs}]({step}/{iters}), '
                f'loss: {current_loss:.4f}, logits_loss: {current_logits_loss:.4f}, '
                f'aux_loss: {current_aux_loss:.4f}, '
                f'lr: {current_lr:.8f}, '
                f'epoch_time: {eta_min:.1f}min'
            )
            if wandb: 
                wandb.log({
                    "loss": current_loss,
                    "logits_loss": current_logits_loss, 
                    "aux_loss": current_aux_loss, 
                    "learning_rate": current_lr, 
                    "epoch_time": eta_min
                })
        
        # 到达指定保存步数时，保存模型（仅主进程）
        if args.save_interval > 0 and (step % args.save_interval == 0 or step == iters - 1):
            if not dist.is_initialized() or dist.get_rank() == 0:
                os.makedirs(args.save_dir, exist_ok=True)  # 确保保存目录存在
                model.eval()
                moe_suffix = '_moe' if lm_config.use_moe else ''
                ckp = f'{args.save_dir}/{args.save_weight}_{lm_config.dim}{moe_suffix}.pth'
                raw_model = model.module if isinstance(model, DistributedDataParallel) else model
                raw_model = getattr(raw_model, '_orig_mod', raw_model)
                state_dict = raw_model.state_dict()
                torch.save({k: v.half().cpu() for k, v in state_dict.items()}, ckp)
                print(f'模型已保存至：{ckp}')
                model.train()
                del state_dict

        del X, Y, loss_mask, res, total_loss, raw_loss

准备完毕，我们尝试一轮长度 1 个 iter 的训练.

In [9]:
iter_per_epoch = len(train_loader)
for epoch in range(args.epochs):
    train_epoch(epoch, train_loader, iter_per_epoch)
print('sft训练完成！')

Epoch:[1/5](1/1), loss: 8.7212, logits_loss: 8.7212, aux_loss: 0.0000, lr: 0.00050225, epoch_time: 0.0min
模型已保存至：./output/minimind_sft_512.pth
Epoch:[2/5](1/1), loss: 7.2350, logits_loss: 7.2350, aux_loss: 0.0000, lr: 0.00037725, epoch_time: 0.0min
模型已保存至：./output/minimind_sft_512.pth
Epoch:[3/5](1/1), loss: 6.2444, logits_loss: 6.2444, aux_loss: 0.0000, lr: 0.00022275, epoch_time: 0.0min
模型已保存至：./output/minimind_sft_512.pth
Epoch:[4/5](1/1), loss: 5.7528, logits_loss: 5.7528, aux_loss: 0.0000, lr: 0.00009775, epoch_time: 0.0min
模型已保存至：./output/minimind_sft_512.pth
Epoch:[5/5](1/1), loss: 5.4913, logits_loss: 5.4913, aux_loss: 0.0000, lr: 0.00005000, epoch_time: 0.0min
模型已保存至：./output/minimind_sft_512.pth
sft训练完成！


In [10]:
del model